In [61]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [62]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5

START_DATE = "20240101"
END_DATE = "20241231"

# Build base name
RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "interm/events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"

In [63]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)

CPU times: user 2.07 ms, sys: 21 ms, total: 23.1 ms
Wall time: 27.9 ms


In [64]:
df.shape

(73782, 60)

In [65]:
df.head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-05 03:25:00,144.8960,144.9415,144.8940,144.9105,2.780660e+12,0.006927,0.000100,0.0145,0.000100,2.788711,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 03:30:00,144.9100,144.9490,144.9060,144.9470,2.220040e+12,0.006398,0.000252,0.0365,0.000252,2.810244,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 03:35:00,144.9465,144.9490,144.9230,144.9240,3.450710e+12,0.006930,-0.000159,-0.0230,-0.000159,2.762497,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 03:40:00,144.9245,144.9460,144.9155,144.9430,2.273510e+12,0.007072,0.000131,0.0190,0.000131,2.784171,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 03:45:00,144.9435,144.9510,144.9355,144.9465,1.925300e+12,0.006585,0.000024,0.0035,0.000024,2.775585,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025


In [66]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_pct_return',
       'close_return', 'close_log_return', 'close_fd_return', 'EMA_5',
       'EMA_20', 'EMA_50', 'EMA_100', 'ATRr_14', 'ATRr_60', 'ATRr_120',
       'ADX_14', 'ADXR_14_2', 'DMP_14', 'DMN_14', 'RSI_14', 'RSI_25', 'RSI_50',
       'BBL_5_2.0_2.0', 'BBM_5_2.0_2.0', 'BBU_5_2.0_2.0', 'BBB_5_2.0_2.0',
       'BBP_5_2.0_2.0', 'BBL_10_2.0_2.0', 'BBM_10_2.0_2.0', 'BBU_10_2.0_2.0',
       'BBB_10_2.0_2.0', 'BBP_10_2.0_2.0', 'BBL_15_2.0_2.0', 'BBM_15_2.0_2.0',
       'BBU_15_2.0_2.0', 'BBB_15_2.0_2.0', 'BBP_15_2.0_2.0', 'BBL_20_2.0_2.0',
       'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0',
       'MACD_8_17_9', 'MACDh_8_17_9', 'MACDs_8_17_9', 'unix_time', 'hour',
       'dow', 'dom', 'month', 'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
       'dom_sin', 'dom_cos', 'month_sin', 'month_cos'],
      dtype='object')

## Apply CUSUM Filter

## Calculate CUSUM Threshold

In [67]:
import numpy as np

In [68]:
return_std = np.log(df['close']).diff().iloc[1:].std()
return_std * 3

np.float64(0.0011693569066543452)

In [74]:
THRESHOLD = 0.001
EVENT_NAME = f'CUSUM_{THRESHOLD}'
EVENT_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

## Get Events Start Time (t0)

In [75]:
import pandas as pd
from tqdm import tqdm

def cusum_filter(closes: pd.Series, threshold: float) -> pd.DatetimeIndex:
    # get tEvents using CUSUM filter
    ret = np.log(closes).diff().iloc[1:]
    values = ret.values
    timestamps = ret.index

    t_events_mask = np.zeros_like(values, dtype=bool)

    cum_pos, cum_neg = 0.0, 0.0

    for i in tqdm(range(len(values))):
        cum_pos = max(0.0, cum_pos + values[i])
        cum_neg = min(0.0, cum_neg + values[i])

        if cum_pos > threshold:
            t_events_mask[i] = True
            cum_pos = 0.0
        elif cum_neg < -threshold:
            t_events_mask[i] = True
            cum_neg = 0.0

    return timestamps[t_events_mask]


In [76]:
%%time

t_events = cusum_filter(df['close'], threshold=THRESHOLD)

100%|███████████████████████████████████| 73781/73781 [00:00<00:00, 1518128.87it/s]

CPU times: user 52.9 ms, sys: 5.95 ms, total: 58.8 ms
Wall time: 54.4 ms


In [77]:
t_events

DatetimeIndex(['2024-01-05 05:15:00', '2024-01-05 06:05:00',
               '2024-01-05 06:20:00', '2024-01-05 06:40:00',
               '2024-01-05 07:00:00', '2024-01-05 08:00:00',
               '2024-01-05 08:30:00', '2024-01-05 09:55:00',
               '2024-01-05 10:25:00', '2024-01-05 10:50:00',
               ...
               '2024-12-30 14:45:00', '2024-12-30 15:00:00',
               '2024-12-30 15:20:00', '2024-12-30 15:35:00',
               '2024-12-30 15:40:00', '2024-12-30 16:40:00',
               '2024-12-30 18:00:00', '2024-12-30 19:00:00',
               '2024-12-30 20:30:00', '2024-12-30 23:05:00'],
              dtype='datetime64[ns]', name='timestamp', length=6633, freq=None)

In [78]:
print(EVENT_NAME)
print(EVENT_PATH)

CUSUM_0.001
../data/interm/events/USDJPY-5m-20240101-20241231_CUSUM_0.001.pkl


In [79]:
t_events.to_series().to_pickle(EVENT_PATH)